In [1]:
import pandas as pd
import requests
import zipfile
from datetime import datetime
import database as db
from pymongo import MongoClient
import openpyxl.reader.excel

In [2]:
def find_report():
    url = 'https://visionet.franceagrimer.fr/Pages/OpenDocument.aspx?fileurl=SeriesChronologiques%2fproductions%20vegetales%2fgrandes%20cultures%2fetats%20des%20cultures%2fSCR-GRC-CEREOBS_M_depuis_2015-A24.xlsx&telechargersanscomptage=oui'
    url2 = 'https://visionet.franceagrimer.fr/Pages/OpenDocument.aspx?fileurl=SeriesChronologiques%2fproductions%20vegetales%2fgrandes%20cultures%2fetats%20des%20cultures%2fSCR-GRC-CEREOBS_CP_depuis_2015-A24.xlsx&telechargersanscomptage=oui'
    try:
        r = requests.get(url)
        r.raise_for_status()
        with open("data/mais.xlsx", "wb") as f:
            f.write(r.content)
        print("File maïs downloaded successfully!")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading the file: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

    try:
        r = requests.get(url2)
        r.raise_for_status()
        with open("data/cereales.xlsx", "wb") as f:
            f.write(r.content)
        print("File céréales downloaded successfully!")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading the file: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [3]:
def monday_of_week(year, week):
    first_day_of_year = pd.to_datetime(f'{year}-01-01')
    first_monday = first_day_of_year - pd.Timedelta(days=first_day_of_year.dayofweek)
    monday_of_given_week = first_monday + pd.Timedelta(weeks=week-1)
    
    return monday_of_given_week

In [4]:
def prep_dataframe():
    find_report()
    dfMais = pd.read_excel('data/mais.xlsx', sheet_name='Données régions', skiprows=5)
    dfMais = dfMais.drop('Unnamed: 0', axis=1)
    dfMais['Week'] = dfMais['Semaine'].str.extract(r'S(\d+)')[0].astype(int)
    dfMais['Year'] = dfMais['Semaine'].str.extract(r'(\d+)-S\d+')[0].astype(int)
    dfMais['Date'] = dfMais.apply(lambda row: monday_of_week(row['Year'], row['Week']), axis=1)

    dfBle = pd.read_excel('data/cereales.xlsx', sheet_name='Données régions', skiprows=5)
    dfBle = dfBle[(dfBle['Culture'] == 'Blé dur') | (dfBle['Culture'] == 'Blé tendre')]
    dfBle = dfBle.drop('Unnamed: 0', axis=1)
    dfBle['Week'] = dfBle['Semaine'].str.extract(r'S(\d+)')[0].astype(int)
    dfBle['Year'] = dfBle['Semaine'].str.extract(r'(\d+)-S\d+')[0].astype(int)
    dfBle['Date'] = dfBle.apply(lambda row: monday_of_week(row['Year'], row['Week']), axis=1)
    return pd.concat([dfMais, dfBle])

In [5]:
x = prep_dataframe()
x

File maïs downloaded successfully!
File céréales downloaded successfully!


,Culture,Région,Semaine,Semis,Levée,6/8 feuilles visibles,Floraison femelle,Humidité du grain 50%,Récolte,Très mauvaises,...,Assez bonnes,Bonnes,Très bonnes,Week,Year,Date,Début tallage,Épi 1 cm,2 noeuds,Épiaison
0,Maïs grain,Auvergne-Rhône-Alpes,2015-S13,0.00,0.00,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,13,2015,2015-03-23,NaN,NaN,NaN,NaN
1,Maïs grain,Auvergne-Rhône-Alpes,2015-S14,0.78,0.00,0.0,0.0,0.0,0.0,NaN,...,NaN,100.00,NaN,14,2015,2015-03-30,NaN,NaN,NaN,NaN
2,Maïs grain,Auvergne-Rhône-Alpes,2015-S15,16.47,0.00,0.0,0.0,0.0,0.0,NaN,...,NaN,100.00,NaN,15,2015,2015-04-06,NaN,NaN,NaN,NaN
3,Maïs grain,Auvergne-Rhône-Alpes,2015-S16,63.70,13.04,0.0,0.0,0.0,0.0,NaN,...,NaN,100.00,NaN,16,2015,2015-04-13,NaN,NaN,NaN,NaN
4,Maïs grain,Auvergne-Rhône-Alpes,2015-S17,81.79,50.42,0.0,0.0,0.0,0.0,NaN,...,4.17,94.44,1.39,17,2015,2015-04-20,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7892,Blé tendre,Provence-Alpes-Côte d'Azur,2024-S16,100.00,100.00,NaN,NaN,NaN,0.0,NaN,...,3.60,54.90,31.32,16,2024,2024-04-15,100.0,100.0,91.86,7.09
7893,Blé tendre,Provence-Alpes-Côte d'Azur,2024-S17,100.00,100.00,NaN,NaN,NaN,0.0,NaN,...,7.20,30.48,62.31,17,2024,2024-04-22,100.0,100.0,100.00,18.84
7894,Blé tendre,Provence-Alpes-Côte d'Azur,2024-S18,100.00,100.00,NaN,NaN,NaN,0.0,NaN,...,8.37,29.32,62.31,18,2024,2024-04-29,100.0,100.0,100.00,46.11
7895,Blé tendre,Provence-Alpes-Côte d'Azur,2024-S19,100.00,100.00,NaN,NaN,NaN,0.0,NaN,...,8.37,29.32,62.31,19,2024,2024-05-06,100.0,100.0,100.00,63.84


In [25]:
grouped_means = x.groupby(['Semaine', 'Culture']).agg({
    'Semis': 'mean',
    'Levée': 'mean',
    'Début tallage': 'mean',
    'Épi 1 cm': 'mean',
    '2 noeuds': 'mean',
    'Épiaison': 'mean',
    '6/8 feuilles visibles': 'mean',
    'Floraison femelle': 'mean',
    'Humidité du grain 50%': 'mean',
    'Récolte': 'mean',
    'Très mauvaises': 'mean',
    'Mauvaises': 'mean',
    'Assez bonnes': 'mean',
    'Bonnes': 'mean',
    'Très bonnes': 'mean'
}).reset_index()
grouped_means['Région'] = 'Moyenne France'
grouped_means['Week'] = grouped_means['Semaine'].str.extract(r'S(\d+)')[0].astype(int)
grouped_means['Year'] = grouped_means['Semaine'].str.extract(r'(\d+)-S\d+')[0].astype(int)
grouped_means['Date'] = grouped_means.apply(lambda row: monday_of_week(row['Year'], row['Week']), axis=1)

df = pd.concat([x, grouped_means]).reset_index(drop=True)
grouped_means

,Semaine,Culture,Semis,Levée,Début tallage,Épi 1 cm,2 noeuds,Épiaison,6/8 feuilles visibles,Floraison femelle,...,Récolte,Très mauvaises,Mauvaises,Assez bonnes,Bonnes,Très bonnes,Région,Week,Year,Date
0,2014-S36,Blé tendre,0.000000,0.000000,0.0,0.0,0.000000,0.000000,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,Moyenne France,36,2014,2014-09-01
1,2014-S37,Blé tendre,0.030000,0.000000,0.0,0.0,0.000000,0.000000,NaN,NaN,...,0.0,NaN,NaN,NaN,100.000000,NaN,Moyenne France,37,2014,2014-09-08
2,2014-S38,Blé tendre,0.968750,0.000000,0.0,0.0,0.000000,0.000000,NaN,NaN,...,0.0,NaN,NaN,9.990000,52.936667,43.730000,Moyenne France,38,2014,2014-09-15
3,2014-S39,Blé tendre,6.112000,0.313000,0.0,0.0,0.000000,0.000000,NaN,NaN,...,0.0,NaN,NaN,7.850000,71.897500,40.254000,Moyenne France,39,2014,2014-09-22
4,2014-S40,Blé dur,0.000000,0.000000,0.0,0.0,0.000000,0.000000,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,Moyenne France,40,2014,2014-09-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329,2024-S19,Blé tendre,100.000000,100.000000,100.0,100.0,99.872500,50.122500,NaN,NaN,...,0.0,2.632857,12.050909,24.690833,54.466667,11.007778,Moyenne France,19,2024,2024-05-06
1330,2024-S19,Maïs grain,73.978889,34.504444,NaN,NaN,NaN,NaN,0.784444,0.0,...,0.0,NaN,5.212500,11.051250,87.221111,1.910000,Moyenne France,19,2024,2024-05-06
1331,2024-S20,Blé dur,100.000000,100.000000,100.0,100.0,94.603333,78.090000,NaN,NaN,...,0.0,3.560000,8.855000,21.106667,65.106667,13.390000,Moyenne France,20,2024,2024-05-13
1332,2024-S20,Blé tendre,100.000000,100.000000,100.0,100.0,100.000000,74.441667,NaN,NaN,...,0.0,2.296250,12.750909,24.699167,53.975000,10.806667,Moyenne France,20,2024,2024-05-13


In [30]:
df = df.sort_values(by='Date')
df

,Culture,Région,Semaine,Semis,Levée,6/8 feuilles visibles,Floraison femelle,Humidité du grain 50%,Récolte,Très mauvaises,...,Assez bonnes,Bonnes,Très bonnes,Week,Year,Date,Début tallage,Épi 1 cm,2 noeuds,Épiaison
11430,Blé tendre,Moyenne France,2014-S36,0.000000,0.000000,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,36,2014,2014-09-01,0.0,0.0,0.00,0.00
7844,Blé tendre,Grand-Est,2014-S36,0.000000,0.000000,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,36,2014,2014-09-01,0.0,0.0,0.00,0.00
6511,Blé tendre,Bourgogne-Franche-Comté,2014-S37,0.000000,0.000000,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,37,2014,2014-09-08,0.0,0.0,0.00,0.00
11431,Blé tendre,Moyenne France,2014-S37,0.030000,0.000000,NaN,NaN,NaN,0.0,NaN,...,NaN,100.000000,NaN,37,2014,2014-09-08,0.0,0.0,0.00,0.00
8308,Blé tendre,Hauts-de-France,2014-S37,0.000000,0.000000,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,37,2014,2014-09-08,0.0,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4370,Blé dur,Centre-Val de Loire,2024-S20,100.000000,100.000000,NaN,NaN,NaN,0.0,NaN,...,21.82000,63.150000,NaN,20,2024,2024-05-13,100.0,100.0,91.62,67.50
10541,Blé tendre,Occitanie,2024-S20,100.000000,100.000000,NaN,NaN,NaN,0.0,NaN,...,41.92000,46.790000,NaN,20,2024,2024-05-13,100.0,100.0,100.00,88.37
5228,Blé dur,Occitanie,2024-S20,100.000000,100.000000,NaN,NaN,NaN,0.0,0.470,...,35.55000,54.460000,1.900,20,2024,2024-05-13,100.0,100.0,100.00,91.06
10092,Blé tendre,Nouvelle-Aquitaine,2024-S20,100.000000,100.000000,NaN,NaN,NaN,0.0,2.260,...,34.91000,39.210000,1.470,20,2024,2024-05-13,100.0,100.0,100.00,96.95


In [31]:
dbname = db.get_database()
collection_name = dbname["new_developpement"]
data = df.to_dict('records')
r = str(collection_name.insert_many(data))
print(r)